### Source 信息修改
---
我们从GEO数据库得到的数据描述信息（source字段）描述了数据的来源信息和处理信息，source字段在数据筛选以及后续的数据分析和展示都起到了很大的作用，但source字段的描述信息很长，不利于展示（如下图1）。参考了SRA数据库的数据描述方法（例 [GSE107580](https://www.ncbi.nlm.nih.gov/Traces/study/?acc=GSE107580&o=acc_s%3Aa))，我将source字段进行了切割和整理，将source字段分割为几个不同的字段分别进行描述（如下图2）。

![img_01](https://cdn.jsdelivr.net/gh/zxzhu/myPic/img/source_img_01.png)
<center>图1 GSE107580 source字段</center>

![img_02](https://cdn.jsdelivr.net/gh/zxzhu/myPic/img/source_img_02.png)
<center>图2 GSE107580 修改后的source字段</center>

**为了**方便后续对source字段的提取，我将每个GSE的GSM对应的source以json格式储存。存储格式为：  
{  
    gse1: {gsm1: {features: {key1: value1, key2: value2 ...}, cancer: value}, cancer_flag: 0 or 1},  
    gse2: {...},...  
}  
以GSE107580为例：  

In [3]:
import json
with open('./sourceInfo.json', 'r') as f:
    aDict = json.load(f)
aDict['GSE107580'].keys()                 # json 文件包含所有的 GSE，每个GSE包含其对应的所有GSM和一个 'cancer_flag' 字段

dict_keys(['GSM2871668', 'GSM2871669', 'GSM2871670', 'GSM2871671', 'GSM2871672', 'GSM2871673', 'GSM3557745', 'GSM3557746', 'cancer_flag'])

In [4]:
aDict['GSE107580']['GSM2871668']            # 每个GSM包含'features'字段（代表source 信息）以及'cancer'字段（代表癌症类型，''为没有）

{'features': {'source_name': 'Sai cells',
  'cell_type': 'fetal-derived neural progenitor cells',
  'vector': 'LV.gRNA.CAS9GFP',
  'grna_target': 'LacZ',
  'viral_vector': ''},
 'cancer': ''}

In [6]:
aDict['GSE107580']['cancer_flag']             # 每个GSE除了含有GSM外，还包含一个 'cancer_flag' 字段，1代表含有癌症数据，0则没有。

0

### 人类数据情况
---
对新收集的人类数据，在分割source字段信息后，我对所有的GSM的癌症信息进行了注释。最后得到如下结果：

In [10]:
# 查看具有癌症数据的GSE
gseList = []
for gse in aDict.keys():
    if aDict[gse]['cancer_flag'] > 0:
        gseList.append(gse)
print(gseList)
print(len(gseList))

['GSE109085', 'GSE109317', 'GSE113946', 'GSE118093', 'GSE122254', 'GSE124610', 'GSE127286', 'GSE127301', 'GSE127304', 'GSE127318', 'GSE127331', 'GSE130591', 'GSE131626', 'GSE27584', 'GSE58695', 'GSE67205', 'GSE80808', 'GSE81541', 'GSE81779', 'GSE82045', 'GSE94331', 'GSE97693', 'GSE98732']
23


**共计23个GSE数据集含有癌症数据。**

In [13]:
# 统计癌症类型和数量
import pandas as pd
import json
import os

dfList = []
for i in os.listdir('human/results/'):
    gse_df = pd.read_csv(os.path.join('human/results/', i))
    gse_df = gse_df[['GEO', 'cancer']]
    gse_df = gse_df.fillna('')
    dfList.append(gse_df)
df = pd.concat(dfList)
with open('./sourceInfo.json', 'r') as f:
    aDict = json.load(f)
    
download_df = pd.read_excel('human/downloads/Homo_sapiens.xlsx')
result_df = pd.merge(download_df, df, how='left').fillna('')

In [16]:
cancerDict = {}
for i in set(result_df['cancer']):
    cancerDict[i] = sum(result_df['cancer']==i)
cancerDict.pop('')
for k, v in cancerDict.items():
    print(k, ':', v)
print(sum(cancerDict.values()))

non-small cell lung cancer : 2
B cell lymphoma : 2
renal carcinoma : 1
colon cancer : 4
glioblastoma : 1
breast cancer : 15
Pancreatic cancer : 6
cervical cancer : 3
HCT116 : 2
prostate cancer : 4
Hep-2 : 18
lung cancer : 6
ovarian cancer : 4
liver cancer : 2
HeLa : 1
leukemia : 1
chronic lymphocytic leukemia : 346
neuroblastoma : 7
425


共计425个GSM是癌症数据。chronic lymphocytic leukemia(慢性淋巴细胞白血病特别多)，看了一下是GSE109085数据集，多为RNA-Seq测序，应该是单细胞测序。

In [17]:
sum((result_df.Library_strategy == 'RNA-Seq') & (result_df['cancer'] != ''))      # RNA-Seq癌症数据

352

In [19]:
sum((result_df.Library_strategy == 'Bisulfite-Seq') & (result_df['cancer'] != ''))   # BS-Seq癌症数据

73